In [22]:
from adam_core.orbits.query import query_sbdb
from adam_core.constants import Constants as C
from adam_core.coordinates import CartesianCoordinates, CoordinateCovariances

orbits = query_sbdb(["2024 YR4"])

In [23]:
from typing import Optional

import numpy as np

from adam_core.coordinates import OriginCodes
from adam_core.orbits import Orbits
from adam_core.propagator import Propagator
from adam_core.time import Timestamp
from adam_core.utils.spice import get_perturber_state

try:
    import plotly.graph_objects as go
except ImportError:
    raise ImportError("Please install adam_core[plots] to use this feature.")


def plot_orbit(
    orbit: Orbits, propagator: Propagator, start_time: Optional[Timestamp] = None
) -> None:
    """
    Plot an orbit.
    """
    # Get the period so we know how long to propagate in cartesian space
    keplerian = orbit.coordinates.to_keplerian()

    period_index = int(np.argmax(keplerian.P))
    period = keplerian.P[period_index]

    if start_time is None:
        start_time = orbit.coordinates.time[period_index]

    # Add the period in days to the orbit.coordinates.time
    propagation_end_date = (
        start_time.add_days(np.ceil(period).astype(int)).mjd()[0].as_py()
    )

    sample_dates_mjd = np.arange(
        start_time.mjd()[0].as_py(), propagation_end_date + 5, 5
    )
    sample_dates = Timestamp.from_mjd(sample_dates_mjd, scale=start_time.scale)

    # Propagate the orbit in cartesian space
    propagated_orbits = propagator.propagate_orbits(orbit, sample_dates)

    max_r = np.max(np.abs(propagated_orbits.coordinates.r))

    # Give a default max_r if the orbits are smaller than the inner planets
    max_r = np.max((max_r, 4))

    # Render the planets conditionally based on max r of the propagated orbits
    conditional_planet_distances = (
        ("Jupiter", 5, OriginCodes.JUPITER_BARYCENTER),
        ("Saturn", 10, OriginCodes.SATURN_BARYCENTER),
        ("Uranus", 15, OriginCodes.URANUS_BARYCENTER),
        ("Neptune", 20, OriginCodes.NEPTUNE_BARYCENTER),
    )

    planet_states = {
        "Earth": get_perturber_state(OriginCodes.EARTH, sample_dates),
        "Mars": get_perturber_state(OriginCodes.MARS_BARYCENTER, sample_dates),
        "Venus": get_perturber_state(OriginCodes.VENUS, sample_dates),
        "Mercury": get_perturber_state(OriginCodes.MERCURY, sample_dates),
    }

    for planet, distance, origin_code in conditional_planet_distances:
        if distance < max_r:
            planet_states[planet] = get_perturber_state(origin_code, sample_dates)

    traces = []

    # Create figure with black background and appropriate styling
    fig = go.Figure(
        layout=dict(
            paper_bgcolor="black",
            plot_bgcolor="black",
            scene=dict(
                bgcolor="black",
            ),
        )
    )

    # Define a color scheme for planets with transparency
    planet_colors = {
        "Mercury": "rgba(160, 82, 45, 0.6)",  # Brown with alpha
        "Venus": "rgba(218, 165, 32, 0.6)",  # Goldenrod with alpha
        "Earth": "rgba(65, 105, 225, 0.6)",  # Royal Blue with alpha
        "Mars": "rgba(205, 92, 92, 0.6)",  # Indian Red with alpha
        "Jupiter": "rgba(222, 184, 135, 0.6)",  # Burlywood with alpha
        "Saturn": "rgba(244, 164, 96, 0.6)",  # Sandy Brown with alpha
        "Uranus": "rgba(135, 206, 235, 0.6)",  # Sky Blue with alpha
        "Neptune": "rgba(30, 144, 255, 0.6)",  # Dodger Blue with alpha
    }

    for planet, state in planet_states.items():
        traces.append(
            go.Scatter3d(
                x=state.x,
                y=state.y,
                z=state.z,
                mode="lines",
                name=planet,
                line=dict(width=1, color=planet_colors[planet]),
            )
        )

    # Use bright green for all non-planet orbits
    orbit_color = "#00FF00"  # Bright green

    for orbit_id in orbit.orbit_id.unique():
        propagated_orbit = propagated_orbits.select("orbit_id", orbit_id)
        cartesian = propagated_orbit.coordinates
        isot_time = propagated_orbit.coordinates.time.to_astropy().isot
        traces.append(
            go.Scatter3d(
                x=cartesian.x,
                y=cartesian.y,
                z=cartesian.z,
                mode="lines",
                name=f"{propagated_orbit.orbit_id[0].as_py()} {propagated_orbit.object_id[0].as_py()}",
                hovertext=isot_time,
                line=dict(width=2, color=orbit_color),
            )
        )

    # For now we are always heliocentric, so plot the sun as a sphere
    traces.append(
        go.Scatter3d(
            x=[0],
            y=[0],
            z=[0],
            mode="markers",
            name="Sun",
            marker=dict(size=1, color="yellow"),
        )
    )

    for trace in traces:
        fig.add_trace(trace)

    max_r_padded = max_r * 1.1
    fig.update(
        layout=dict(
            scene=dict(
                xaxis=dict(
                    range=[-max_r_padded, max_r_padded],
                    gridcolor="rgba(128, 128, 128, 0.2)",
                    showbackground=False,
                    color="white",
                ),
                yaxis=dict(
                    range=[-max_r_padded, max_r_padded],
                    gridcolor="rgba(128, 128, 128, 0.2)",
                    showbackground=False,
                    color="white",
                ),
                zaxis=dict(
                    range=[-max_r_padded, max_r_padded],
                    gridcolor="rgba(128, 128, 128, 0.2)",
                    showbackground=False,
                    color="white",
                ),
                xaxis_title="x [au]",
                yaxis_title="y [au]",
                zaxis_title="z [au]",
            ),
            font=dict(color="white"),  # Make all text white
        )
    )
    fig.show()

In [24]:
from adam_assist import ASSISTPropagator

propagator = ASSISTPropagator(
    min_dt=1e-9,
    initial_dt=1e-6,
    epsilon=1e-9,
    adaptive_mode=1
)

In [25]:
plot_orbit(orbits, propagator)

In [26]:
from adam_core.orbits import VariantOrbits

orbits = VariantOrbits.create(orbits, method="monte-carlo", num_samples=10000)

In [27]:
propagated_variants, impacts = propagator.detect_impacts(orbits, 10*365, max_processes=10)

In [28]:
print(impacts.to_dataframe())

    orbit_id     distance  coordinates.x  coordinates.y  coordinates.z  \
0      00000  5994.486084      -0.013974       0.983495      -0.000051   
1      00000  6134.421551      -0.013994       0.983497      -0.000051   
2      00000  6341.776803      -0.013972       0.983497      -0.000051   
3      00000  6041.232021      -0.014120       0.983532      -0.000059   
4      00000  6249.420102      -0.014015       0.983502      -0.000052   
..       ...          ...            ...            ...            ...   
97     00000  6313.743799      -0.013983       0.983495      -0.000051   
98     00000  6307.339051      -0.013972       0.983497      -0.000051   
99     00000  6275.373037      -0.014005       0.983513      -0.000055   
100    00000  6103.918401      -0.014060       0.983515      -0.000055   
101    00000  6205.409841      -0.013985       0.983495      -0.000051   

     coordinates.vx  coordinates.vy  coordinates.vz  coordinates.time.days  \
0         -0.021499        0.0087

In [29]:
#convert to geocentric
from adam_core.coordinates import transform_coordinates
from adam_core.coordinates.origin import OriginCodes
import spiceypy as sp
import quivr as qv

geocentric_impact_coords = transform_coordinates(impacts.coordinates, origin_out=OriginCodes.EARTH, frame_out="equatorial")
print(geocentric_impact_coords.to_dataframe())

# Lets loop through the coordinates and rotate them to an Earth body-fixed frame 
# This is a time varying rotation
# sp.furnsh(DEFAULT_KERNELS)

coords_rotated = CartesianCoordinates.empty()
for coord in geocentric_impact_coords:

     unique_time_et = coord.time.et().to_numpy()[0]

     rotation_matrix_6x6 = np.zeros((6, 6))
     # In SPICE parlance J2000 and EME2000 are the same
     rotation_matrix = sp.pxform("J2000", "ITRF93", unique_time_et)
     rotation_matrix_6x6[:3, :3] = rotation_matrix
     rotation_matrix_6x6[3:, 3:] = rotation_matrix

     coords_rotated = qv.concatenate([coords_rotated, coord.rotate(rotation_matrix_6x6, "ITRF93")])

print(coords_rotated)

# Convert to spherical coordinates
spherical_coords = coords_rotated.to_spherical()
print(spherical_coords.to_dataframe())

            x             y         z        vx        vy        vz  \
0   -0.000005 -3.964440e-05  0.000003 -0.004019  0.009166  0.001677   
1    0.000018 -3.667101e-05  0.000004 -0.004681  0.008798  0.001633   
2   -0.000018 -3.825870e-05  0.000003 -0.003614  0.009200  0.001688   
3    0.000039  5.273928e-07  0.000012 -0.005985  0.008063  0.001344   
4    0.000027 -3.120527e-05  0.000005 -0.004947  0.008611  0.001597   
..        ...           ...       ...       ...       ...       ...   
97   0.000014 -3.961078e-05  0.000003 -0.004533  0.008801  0.001652   
98  -0.000018 -3.825149e-05  0.000003 -0.003629  0.009206  0.001687   
99  -0.000036 -2.152498e-05  0.000006 -0.002745  0.009530  0.001620   
100  0.000036 -1.781630e-05  0.000008 -0.005411  0.008405  0.001506   
101  0.000014 -3.906443e-05  0.000003 -0.004536  0.008842  0.001650   

     time.days      time.nanos  \
0        63588  50055360281467   
1        63588  50270349942148   
2        63588  49983094851673   
3        63

In [30]:
from adam_core.constants import Constants as C

print(spherical_coords.rho.to_numpy()/C.R_EARTH_POLAR)

[0.9430108  0.9650245  0.99764417 0.95036462 0.98311529 0.99164599
 0.9271497  0.974169   0.97219361 0.96990532 0.96988894 0.93890468
 0.93267661 0.9824113  0.94884152 0.94502907 0.99938879 0.98178172
 0.99546737 0.97054458 0.95435957 0.99852393 0.94574571 0.9982315
 0.97289367 0.97556926 0.93094654 0.95312595 0.94747953 0.99194556
 0.95506512 0.96561159 0.93722011 0.93779757 0.99964239 0.98938129
 0.96397884 0.93918911 0.95516545 0.97615776 1.00051145 0.93904753
 0.98283185 0.98419634 0.96488439 0.94160955 0.9652051  0.99824363
 0.96835389 0.9362988  0.96659133 0.97974379 0.97482243 1.00083174
 0.95339017 0.99729353 0.99797198 0.99736086 0.97692992 0.9511569
 1.00241564 0.99264969 0.98469599 0.98088964 0.97354754 0.97416004
 0.97773271 0.9494636  0.93023744 0.94692592 0.93119807 0.94018889
 0.96656325 0.96161471 0.9636291  0.94071361 0.98614804 0.9763231
 0.99759165 0.99691952 0.9825873  0.96671677 0.96838501 0.95418672
 0.99654454 0.93268261 0.96408987 0.99442751 0.95720732 0.9669512

In [38]:
#filter propagated_variants for those that do not have an impact
import pyarrow.compute as pc

impacting_ids = impacts.variant_id.unique()

non_impacting_variants = propagated_variants.apply_mask(pc.invert(pc.is_in(propagated_variants.variant_id, impacting_ids)))

print(non_impacting_variants.to_dataframe())

mean_impact_times = pc.mean(impacts.coordinates.time.mjd())
print(impacts.coordinates.time.scale)



     orbit_id   object_id variant_id  weights  weights_cov  coordinates.x  \
0       00000  (2024 YR4)        600   0.0001       0.0001      -2.398743   
1       00000  (2024 YR4)        601   0.0001       0.0001      -2.675800   
2       00000  (2024 YR4)        603   0.0001       0.0001      -2.675050   
3       00000  (2024 YR4)        604   0.0001       0.0001      -2.290307   
4       00000  (2024 YR4)        605   0.0001       0.0001      -2.678785   
...       ...         ...        ...      ...          ...            ...   
9893    00000  (2024 YR4)       9795   0.0001       0.0001      -2.899961   
9894    00000  (2024 YR4)       9796   0.0001       0.0001      -2.395710   
9895    00000  (2024 YR4)       9797   0.0001       0.0001      -3.168467   
9896    00000  (2024 YR4)       9798   0.0001       0.0001      -2.681020   
9897    00000  (2024 YR4)       9799   0.0001       0.0001       0.706507   

      coordinates.y  coordinates.z  coordinates.vx  coordinates.vy  \
0    

In [39]:
#propagate the non-impacting variants to the mean impact time
mean_impact_times = Timestamp.from_mjd([mean_impact_times], scale="tdb")
non_impacting_variants_propagated = propagator.propagate_orbits(non_impacting_variants, mean_impact_times, max_processes=10)

print(non_impacting_variants_propagated.to_dataframe())


KeyboardInterrupt: 

In [ ]:

def rotate_to_ITRF93(coords):
    coords_rotated = CartesianCoordinates.empty()
    for coord in geocentric_impact_coords:

        unique_time_et = coord.time.et().to_numpy()[0]

        rotation_matrix_6x6 = np.zeros((6, 6))
        # In SPICE parlance J2000 and EME2000 are the same
        rotation_matrix = sp.pxform("J2000", "ITRF93", unique_time_et)
        rotation_matrix_6x6[:3, :3] = rotation_matrix
        rotation_matrix_6x6[3:, 3:] = rotation_matrix

        coords_rotated = qv.concatenate([coords_rotated, coord.rotate(rotation_matrix_6x6, "ITRF93")])
    return coords_rotated

In [ ]:
geocentric_non_impact_coords = transform_coordinates(non_impacting_variants_propagated.coordinates, origin_out=OriginCodes.EARTH, frame_out="equatorial")
non_impacting_points_rotated = rotate_to_ITRF93(geocentric_non_impact_coords)

print(non_impacting_points_rotated.to_dataframe())


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Create a sphere for Earth
def create_earth_sphere(radius=1, resolution=50):
    phi = np.linspace(0, 2*np.pi, resolution)
    theta = np.linspace(-np.pi/2, np.pi/2, resolution)
    phi, theta = np.meshgrid(phi, theta)
    
    x = radius * np.cos(theta) * np.cos(phi)
    y = radius * np.cos(theta) * np.sin(phi)
    z = radius * np.sin(theta)
    
    return x, y, z

# Create figure
fig = go.Figure()

# Add Earth as a surface
x_earth, y_earth, z_earth = create_earth_sphere()
fig.add_trace(go.Surface(
    x=x_earth, y=y_earth, z=z_earth,
    colorscale=[[0, 'rgb(0, 0, 100)'], [1, 'rgb(100, 100, 100)']],
    showscale=False,
    name='Earth',
    opacity=0.8
))

# Convert impact points from lat/lon to 3D coordinates
impact_x = np.cos(np.radians(spherical_coords.lat.to_numpy())) * np.cos(np.radians(spherical_coords.lon.to_numpy()))
impact_y = np.cos(np.radians(spherical_coords.lat.to_numpy())) * np.sin(np.radians(spherical_coords.lon.to_numpy()))
impact_z = np.sin(np.radians(spherical_coords.lat.to_numpy()))

# Add impact points
fig.add_trace(go.Scatter3d(
    x=impact_x,
    y=impact_y,
    z=impact_z,
    mode='markers',
    marker=dict(
        size=5,
        color='red',
        symbol='circle',
    ),
    name='Impact Points',
))

# Add non-impacting points
# Get the final state of non-impacting variants and scale by Earth radius
non_impacting_coords = non_impacting_variants_propagated.coordinates
scale_factor = 1/C.R_EARTH_POLAR

fig.add_trace(go.Scatter3d(
    x=non_impacting_coords.x.to_numpy() * scale_factor,
    y=non_impacting_coords.y.to_numpy() * scale_factor,
    z=non_impacting_coords.z.to_numpy() * scale_factor,
    mode='markers',
    marker=dict(
        size=3,
        color='blue',
        opacity=0.6,
    ),
    name='Non-impacting Points',
))

# Update layout
fig.update_layout(
    scene=dict(
        aspectmode='data',
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.5)
        ),
        xaxis=dict(
            range=[-5, 5],
            showbackground=False,
            gridcolor='white',
            showgrid=True,
            zeroline=True,
            zerolinecolor='white',
        ),
        yaxis=dict(
            range=[-5, 5],
            showbackground=False,
            gridcolor='white',
            showgrid=True,
            zeroline=True,
            zerolinecolor='white',
        ),
        zaxis=dict(
            range=[-5, 5],
            showbackground=False,
            gridcolor='white',
            showgrid=True,
            zeroline=True,
            zerolinecolor='white',
        ),
    ),
    paper_bgcolor='black',
    plot_bgcolor='black',
    showlegend=True,
    legend=dict(
        font=dict(color='white'),
    ),
    font=dict(color='white'),
)

fig.show()

In [31]:
# Create the Earth globe visualization with impact points
import plotly.graph_objects as go

# Create figure
fig = go.Figure()

# Add the base map/globe
fig.add_trace(go.Scattergeo(
    mode='none',
    showlegend=False,
))

# Add the impact points
fig.add_trace(go.Scattergeo(
    lon=spherical_coords.lon.to_numpy(),
    lat=spherical_coords.lat.to_numpy(),
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='circle',
    ),
    name='Impact Points',
    hovertext=[f'Lat: {lat:.2f}°, Lon: {lon:.2f}°' 
               for lat, lon in zip(spherical_coords.lat.to_numpy(), 
                                 spherical_coords.lon.to_numpy())],
))

# Update the layout for a nice looking globe
fig.update_layout(
    showlegend=True,
    geo=dict(
        projection_type='orthographic',
        showland=True,
        showcountries=True,
        showocean=True,
        countrycolor='white',
        oceancolor='rgb(0, 0, 100)',
        landcolor='rgb(100, 100, 100)',
        bgcolor='black',
        framecolor='white',
        framewidth=2,
        lataxis=dict(gridcolor='white', gridwidth=0.5),
        lonaxis=dict(gridcolor='white', gridwidth=0.5),
    ),
    paper_bgcolor='black',
    plot_bgcolor='black',
    font=dict(color='white'),
)

# Add buttons to rotate the view
fig.update_layout(
    updatemenus=[dict(
        type='buttons',
        showactive=False,
        buttons=[
            dict(
                args=[{'geo.projection.rotation.lon': 0,
                      'geo.projection.rotation.lat': 0}],
                label='Reset View',
                method='relayout'
            )
        ],
        x=0.9,
        y=0.9,
    )]
)

fig.show()